# Plaka Tespiti - YOLOv8 Eğitimi

Bu notebook, UFPR-ALPR veri seti kullanarak plaka tespiti için bir YOLOv8 modeli eğitimi gerçekleştirir.

## 1. Kurulum ve İçe Aktarmalar

In [1]:
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import torch
from pathlib import Path
import shutil

In [2]:
# src dizinini yola ekle
sys.path.append(os.path.abspath('../src'))

In [3]:
# Özel modülleri içe aktar
from plate_detection import PlateDetector
from preprocessing import preprocess_image_for_plate_detection

import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.7.0+cpu
0.22.0+cpu


## 2. Veri Hazırlama ve Eğitim Parametreleri

UFPR-ALPR veri setini YOLOv8 formatına dönüştürmek ve eğitim için parametreleri belirlemek.

In [4]:
# Yolları tanımla - UFPR-ALPR veri seti için özelleştirildi
dataset_root = '../data/raw/ufpr-alpr'
processed_dataset_root = '../data/processed'

# GPU kullanımı kontrolü
has_gpu = torch.cuda.is_available()
device = '0' if has_gpu else 'cpu'
print(f"Mevcut cihaz: {'GPU' if has_gpu else 'CPU'}")

# Eğitim parametreleri - daha hızlı eğitim için optimize edildi
epochs = 10  # Toplam eğitim adımı (100 yerine)
batch_size = 16  # Toplu iş boyutu
img_size = 720  # Görüntü boyutu (640 yerine) - daha hızlı işlem için

Mevcut cihaz: CPU


## 3. UFPR-ALPR Veri Setini YOLOv8 Formatına Dönüştürme

UFPR-ALPR veri seti, plaka tespiti için özel olarak tasarlanmış bir veri setidir. Bu veri setini YOLOv8 formatına dönüştürmek için `PlateDetector` sınıfındaki `prepare_ufpr_dataset` fonksiyonunu kullanacağız.

Bu fonksiyon şunları yapar:
1. UFPR-ALPR veri setindeki görüntüleri ve etiketleri okur
2. Plaka köşe koordinatlarını YOLO formatına (normalize edilmiş merkez x, merkez y, genişlik, yükseklik) dönüştürür
3. Görüntüleri ve etiketleri YOLOv8'in beklediği klasör yapısına kopyalar
4. Veri seti için YAML yapılandırma dosyası oluşturur

In [5]:
# Çıktı dizinleri oluştur
os.makedirs(processed_dataset_root, exist_ok=True)

In [6]:
# Tespit modülünü başlat
detector = PlateDetector()

# Veri seti yaml yolunu kontrol et
dataset_yaml = os.path.join(processed_dataset_root, 'dataset.yaml')
if not os.path.exists(dataset_yaml):
    print("UFPR-ALPR veri seti hazırlanıyor...")
    prepared_dataset_path = detector.prepare_ufpr_dataset(dataset_root, processed_dataset_root)
    print(f"Veri seti şurada hazırlandı: {prepared_dataset_path}")
else:
    prepared_dataset_path = processed_dataset_root
    print("Mevcut veri seti kullanılıyor...")

UFPR-ALPR veri seti hazırlanıyor...
Veri seti hazırlama başladı...
Toplam görüntü sayısı hesaplanıyor...
Toplam 4500 görüntü işlenecek

training bölümü işleniyor...


training izlemeleri:   5%|▌         | 3/60 [00:06<02:05,  2.20s/it]


İşlenen görüntü: 100/4500 (2.2%)
Geçen süre: 0.1 dakika
Tahmini kalan süre: 5.5 dakika


training izlemeleri:  10%|█         | 6/60 [00:13<01:56,  2.16s/it]


İşlenen görüntü: 200/4500 (4.4%)
Geçen süre: 0.2 dakika
Tahmini kalan süre: 5.3 dakika


...


İşlenen görüntü: 4300/4500 (95.6%)
Geçen süre: 7.8 dakika
Tahmini kalan süre: 0.4 dakika


testing izlemeleri:  93%|█████████▎| 56/60 [03:34<00:16,  4.23s/it]


İşlenen görüntü: 4400/4500 (97.8%)
Geçen süre: 8.0 dakika
Tahmini kalan süre: 0.2 dakika


testing izlemeleri: 100%|██████████| 60/60 [03:49<00:00,  3.83s/it]


İşlenen görüntü: 4500/4500 (100.0%)
Geçen süre: 8.2 dakika
Tahmini kalan süre: 0.0 dakika

Veri seti hazırlama tamamlandı!
Toplam süre: 8.2 dakika
İşlenen toplam görüntü: 4500
Veri seti şurada hazırlandı: ../data/processed


## 4. YOLOv8 Modelini Eğit

In [7]:
import os
print(os.getlogin())

# Modeli eğit
print(f"Model eğitimi başlatılıyor (epochs={epochs}, batch_size={batch_size}, img_size={img_size})...")
model_path = detector.train_custom_model(
    prepared_dataset_path,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size
)
print(f"Eğitim tamamlandı. Model şuraya kaydedildi: {model_path}")

eralperat
Model eğitimi başlatılıyor (epochs=10, batch_size=16, img_size=720)...
YAML dosyası okunabildi: ../data/processed\dataset.yaml
train dizini erişilebilir (2 öğe)
val dizini erişilebilir (2 öğe)
test dizini erişilebilir (2 öğe)
Eğitim çıktıları şuraya kaydedilecek: c:\msworks\APLR_Final\notebooks\runs\train\yolov8_plate_1746982620
YOLOv8n modeli başarıyla yüklendi
Eğitim başlatılıyor: 10 epochs, 16 batch size, 720 image size
Veri seti: ../data/processed
Cihaz: CPU
New https://pypi.org/project/ultralytics/8.3.131 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.127  Python-3.10.11 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i7-1185G7 3.00GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/processed\dataset.yaml, degrees=0.0, deterministic=True,

train: Scanning C:\msworks\APLR_Final\data\processed\train\labels... 1800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1800/1800 [00:32<00:00, 55.01it/s]


train: New cache created: C:\msworks\APLR_Final\data\processed\train\labels.cache
val: Fast image access  (ping: 0.30.1 ms, read: 444.943.6 MB/s, size: 2255.9 KB)


c:\Python310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\msworks\APLR_Final\data\processed\val\labels... 900 images, 0 backgrounds, 0 corrupt: 100%|██████████| 900/900 [00:18<00:00, 49.93it/s]


val: New cache created: C:\msworks\APLR_Final\data\processed\val\labels.cache


...

      10/10         0G     0.5667     0.3761     0.7989          7        736: 100%|██████████| 113/113 [22:16<00:00, 11.82s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [03:00<00:00,  6.22s/it]

                   all        900        900      0.982      0.983      0.994      0.747



10 epochs completed in 4.061 hours.
Optimizer stripped from c:\msworks\APLR_Final\notebooks\runs\train\yolov8_plate_17469826202\weights\last.pt, 6.2MB
Optimizer stripped from c:\msworks\APLR_Final\notebooks\runs\train\yolov8_plate_17469826202\weights\best.pt, 6.2MB

Validating c:\msworks\APLR_Final\notebooks\runs\train\yolov8_plate_17469826202\weights\best.pt...
Ultralytics 8.3.127  Python-3.10.11 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i7-1185G7 3.00GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [02:42<00:00,  5.62s/it]


                   all        900        900      0.982      0.984      0.994      0.747
Speed: 3.2ms preprocess, 107.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to c:\msworks\APLR_Final\notebooks\runs\train\yolov8_plate_17469826202
Eğitim başarıyla tamamlandı. Model kaydedildi: c:\msworks\APLR_Final\notebooks\runs\train\yolov8_plate_17469826202\weights\best.pt
Eğitim tamamlandı. Model şuraya kaydedildi: c:\msworks\APLR_Final\notebooks\runs\train\yolov8_plate_17469826202\weights\best.pt


## 5. Modeli Klasöre Kopyala

In [8]:
import os
print(model_path)
print(os.path.exists(model_path))

# Eğitilmiş modeli models dizinine kopyala
os.makedirs('../models', exist_ok=True)
target_model_path = '../models/yolov8_license_plate.pt'

try:
    shutil.copy2(model_path, target_model_path)
    print(f"Model başarıyla şuraya kopyalandı: {target_model_path}")
except Exception as e:
    print(f"Model kopyalanırken hata oluştu: {str(e)}")

c:\msworks\APLR_Final\notebooks\runs\train\yolov8_plate_17469826202\weights\best.pt
True
Model başarıyla şuraya kopyalandı: ../models/yolov8_license_plate.pt


## 6. Modeli Test Et

In [9]:
def test_model_on_image(model_path, image_path):
    """Eğitilmiş modeli bir görüntü üzerinde test et"""
    print(f"Model şu görüntü üzerinde test ediliyor: {image_path}")
    
    # Modeli yükle
    model = YOLO(model_path)
    
    # Görüntüyü yükle
    image = cv2.imread(image_path)
    if image is None:
        print(f"Hata: {image_path} yüklenemedi")
        return
    
    # Plakaları tespit et
    results = model(image)[0]
    
    # Sonuçları göster
    plt.figure(figsize=(12, 8))
    result_image = results.plot()
    plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    plt.title("Tespit Edilen Plakalar")
    plt.axis('off')
    plt.show()
    
    # Tespit sonuçlarını yazdır
    print(f"Tespit edilen plakaların sayısı: {len(results.boxes)}")
    for i, box in enumerate(results.boxes):
        print(f"Plaka {i+1}: Konum={box.xyxy[0].tolist()}, Güven={box.conf[0].item():.2f}")

In [10]:
# Test et - UFPR-ALPR test setinden bir görüntü kullan
test_image_path = '../data/raw/ufpr-alpr/testing/track0091/track0091[01].png'

# Görüntü yoksa atlama yap
if os.path.exists(test_image_path):
    test_model_on_image(target_model_path, test_image_path)
else:
    print(f"Test görüntüsü bulunamadı: {test_image_path}")
    print("Lütfen test için geçerli bir görüntü yolu belirtin.")

Model şu görüntü üzerinde test ediliyor: ../data/raw/ufpr-alpr/testing/track0091/track0091[01].png

0: 416x736 1 license_plate, 171.1ms
Speed: 5.7ms preprocess, 171.1ms inference, 1.8ms postprocess per image at shape (1, 3, 416, 736)


<Figure size 1200x800 with 1 Axes>

Tespit edilen plakaların sayısı: 1
Plaka 1: Konum=[856.427978515625, 502.34771728515625, 916.5349731445312, 525.538818359375], Güven=0.87
